mlflow ui --backend-store-uri sqlite:///mlflow.db

In [9]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import Lasso
from sklearn.metrics import root_mean_squared_error

def read_dataframe(path):
    df = pd.read_parquet(path)
    df["duration"] = (df.lpep_dropoff_datetime - df.lpep_pickup_datetime).dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    df[["PULocationID", "DOLocationID"]] = df[["PULocationID", "DOLocationID"]].astype(str)
    return df

df_train = read_dataframe("../data/green_tripdata_2025-01.parquet")
df_val = read_dataframe("../data/green_tripdata_2025-02.parquet")

df_train["PU_DO"] = df_train["PULocationID"] + "_" + df_train["DOLocationID"]
df_val["PU_DO"] = df_val["PULocationID"] + "_" + df_val["DOLocationID"]

categorical = ["PU_DO"]
numerical = ["trip_distance"]

dv = DictVectorizer()
X_train = dv.fit_transform(df_train[categorical + numerical].to_dict(orient="records"))
X_val = dv.transform(df_val[categorical + numerical].to_dict(orient="records"))

y_train = df_train["duration"].values
y_val = df_val["duration"].values

In [10]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("class-nyc-taxi-experiment")

with mlflow.start_run(run_name="lasso_alpha_0.1"):
    alpha = 0.1
    mlflow.log_param("alpha", alpha)

    model = Lasso(alpha=alpha)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_val)
    rmse = root_mean_squared_error(y_val, y_pred)

    mlflow.log_metric("rmse", rmse)
    mlflow.sklearn.log_model(model, "model")

    print(f"✅ Run finalizado. RMSE = {rmse:.4f}")

2025/10/16 20:35:06 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/10/16 20:35:07 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/16 20:35:08 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/10/16 20:35:08 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


✅ Run finalizado. RMSE = 8.9926


## Model registry 2

In [11]:
from sklearn.ensemble import GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR
import pickle

import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment(experiment_name="nyc-taxi-model-registry-example")

<Experiment: artifact_location='/Users/msgarcia/Desktop/School/proyecto_2/nyc-taxi-predictions-2025/notebooks/mlruns/2', creation_time=1760498421853, experiment_id='2', last_update_time=1760498421853, lifecycle_stage='active', name='nyc-taxi-model-registry-example', tags={'mlflow.experimentKind': 'custom_model_development'}>

In [12]:
mlflow.sklearn.autolog()
training_dataset = mlflow.data.from_numpy(X_train.data, targets=y_train, name="green_tripdata_2025-01")
validation_dataset = mlflow.data.from_numpy(X_val.data, targets=y_val, name="green_tripdata_2025-02")

In [13]:
models = [
    
    {"model": GradientBoostingRegressor,
     "params": {"n_estimators": 100, "learning_rate": 0.3, "max_depth": 25, "random_state": 42},
     },
    
    {"model": ExtraTreesRegressor,
     "params": {"n_estimators": 100, "max_depth": 15, "random_state": 42},
     },
    
    {"model": LinearSVR,
     "params": {"C": 1.0, "epsilon": 0}, 
     },

]

In [14]:
with mlflow.start_run(run_name="Nested Runs"):
    for model in models:
        
        model_class = model["model"]
        model_name = model_class.__name__
        params = model["params"]
        
        with mlflow.start_run(run_name=model_name,nested=True):
            
            ml_model = model_class(**params)
           
            ml_model.fit(X_train, y_train)
    
            y_pred = ml_model.predict(X_val)
            
            rmse = root_mean_squared_error(y_val, y_pred)
            mlflow.log_metric("rmse", rmse)
            
            !mkdir models
            with open("models/preprocessor.b", "wb") as f_out:
                pickle.dump(dv, f_out)
                
            mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

2025/10/16 20:35:17 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


mkdir: models: File exists


2025/10/16 20:35:23 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


mkdir: models: File exists


/Users/msgarcia/Desktop/School/proyecto_2/nyc-taxi-predictions-2025/.venv/lib/python3.11/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
2025/10/16 20:35:26 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


mkdir: models: File exists


## Registrar modelo 


In [15]:
from sklearn.ensemble import RandomForestRegressor


with mlflow.start_run(run_name="RandomForestRegressor"):
    ml_model = RandomForestRegressor(
        n_estimators=100,
        max_depth=15,
        random_state=42
    )
    
    ml_model.fit(X_train, y_train)
    
    mlflow.sklearn.log_model(
        sk_model=model, 
        artifact_path="model",
        registered_model_name="nyc-taxi-model"
    )
    
    y_pred = ml_model.predict(X_val)
    
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)
    
    # !mkdir models
    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
        
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

2025/10/16 20:35:34 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/10/16 20:35:35 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/16 20:35:35 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2025/10/16 20:35:36 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/10/16 20:35:36 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'nyc-taxi-model' already exists. Creating a new version of this model...
Created version '5' of model 'nyc-taxi-model'.



## Metodo 2 para registrar modelo

In [16]:
"""run_id = input("Ingrese el run_id")
run_uri = f"runs:/{run_id}/model"

result = mlflow.register_model(
    model_uri=run_uri,
    name="nyc-taxi-model"
)"""

'run_id = input("Ingrese el run_id")\nrun_uri = f"runs:/{run_id}/model"\n\nresult = mlflow.register_model(\n    model_uri=run_uri,\n    name="nyc-taxi-model"\n)'

## 3 forma de registrar un modelo 

In [17]:
from mlflow import MlflowClient

client = MlflowClient(tracking_uri="sqlite:///mlflow.db")

In [19]:
# client.create_registered_model(name="nyc-taxi-model")


In [20]:
run_id = input("Ingrese el run_id")
run_uri = f"runs:/{run_id}/model"

result = client.create_model_version(
    name="nyc-taxi-model",
    source=run_uri,
    run_id=run_id
)

## asignar alias

In [44]:
# create "champion" alias for version 1 of model "example-model"
client.set_registered_model_alias(
    name="nyc-taxi-model", 
    alias="champion",
    version=6
)


In [45]:
client.set_registered_model_alias(
    name="nyc-taxi-model", 
    alias="challenger",
    version=1
)

In [24]:
# delete the alias
client.delete_registered_model_alias(
    name="nyc-taxi-model", 
    alias="challenger"
)

In [27]:
# get a model version by alias
print(client.get_model_version_by_alias(
    name="nyc-taxi-model",
    alias="champion"
).version)

6


## Actualizar version del modelo 

In [23]:
client.update_model_version(
    name="nyc-taxi-model",
    version=5,
    description="This model version is a scikit-learn random forest containing 100 decision trees",
)

<ModelVersion: aliases=[], creation_timestamp=1760668536451, current_stage='None', deployment_job_state=None, description=('This model version is a scikit-learn random forest containing 100 decision '
 'trees'), last_updated_timestamp=1760669220172, metrics=None, model_id=None, name='nyc-taxi-model', params=None, run_id='6b2c0b2a086840f8b71ea49c857036f3', run_link=None, source='models:/m-84c8d215009a4e75a48d30943d5fd2c2', status='READY', status_message=None, tags={}, user_id=None, version=5>

## Obtener version del modelo 

In [36]:
import mlflow.pyfunc

model_name = "nyc-taxi-model"
model_version = 6

model_uri = f"models:/{model_name}/{model_version}"

model = mlflow.pyfunc.load_model(
    model_uri=model_uri,
)

model.predict(X_val)

MlflowException: Model does not have the "python_function" flavor

In [34]:
import mlflow.pyfunc

model_name = "nyc-taxi-model"
alias = "champion"

model_uri = f"models:/{model_name}@{alias}"

champion_version = mlflow.sklearn.load_model(
    model_uri=model_uri
)

champion_version.predict(X_val)

AttributeError: 'dict' object has no attribute 'predict'

In [53]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, alias, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}@{alias}")
    y_pred = model.predict(X_test)
    return {"rmse": root_mean_squared_error(y_test, y_pred)}

In [38]:
df = read_dataframe("../data/green_tripdata_2025-03.parquet")


In [41]:
run_id = input("Ingrese el run_id")

client.download_artifacts(
    run_id=run_id, 
    path='preprocessor', 
    dst_path='.'
)

'/Users/msgarcia/Desktop/School/proyecto_2/nyc-taxi-predictions-2025/notebooks/preprocessor'

In [42]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)
X_test = preprocess(df, dv)

In [47]:
target = "duration"
y_test = df[target].values


In [62]:
client.set_registered_model_alias(
    name="nyc-taxi-model", 
    alias="champion",
    version=1
)

client.set_registered_model_alias(
    name="nyc-taxi-model", 
    alias="challenger",
    version=3
)

In [63]:
%time test_model(name="nyc-taxi-model", alias="champion", X_test=X_test, y_test=y_test)
%time test_model(name=model_name, alias="challenger", X_test=X_test, y_test=y_test)



CPU times: user 550 ms, sys: 7.33 ms, total: 558 ms
Wall time: 562 ms
CPU times: user 14.5 ms, sys: 1.4 ms, total: 15.9 ms
Wall time: 17.5 ms


{'rmse': 538.963705186277}